Library

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
# Import metrics baru
from tensorflow.keras.metrics import AUC, Precision, Recall, BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2 # Contoh model pre-trained

# Pastikan BASE_DIR_DATASET sudah didefinisikan dari skrip preprocessing atau definisikan ulang
# Asumsi notebook berada di 'notebooks/' dan dataset di 'dataset/'
import os
import numpy as np # Pastikan numpy diimpor
from sklearn.model_selection import train_test_split # Pastikan train_test_split diimpor
import matplotlib.pyplot as plt # Pastikan matplotlib diimpor

BASE_DIR_DATASET = os.path.abspath(os.path.join(os.getcwd(), '..', 'dataset'))
IMG_SIZE = (224, 224) # Ukuran gambar target untuk model CNN
# Ini adalah definisi LABELS_FINAL yang sama seperti di skrip pembuatan dataset dan preprocessing
LABELS_FINAL = ['battery', 'organik', 'glass', 'cardboard', 'metal', 'paper', 'plastic', 'trash']

Load Data

In [ ]:
print("\n--- 7. Memuat Data yang Sudah Diproses (NumPy Arrays) ---")
try:
    X = np.load(os.path.join(BASE_DIR_DATASET, 'X_data.npy'))
    Y = np.load(os.path.join(BASE_DIR_DATASET, 'Y_labels.npy'))
    print(f"Data X.npy dimuat dengan bentuk: {X.shape}")
    print(f"Data Y.npy dimuat dengan bentuk: {Y.shape}")
except FileNotFoundError:
    print(f"Error: File X_data.npy atau Y_labels.npy tidak ditemukan di {BASE_DIR_DATASET}.")
    print("Pastikan Anda sudah menjalankan bagian Preprocessing dan konversi NumPy array.")
    exit()

Split Data Train/Validation/Test

In [ ]:
print("\n--- 8. Membagi Data menjadi Train, Validation, dan Test Set ---")
# Split awal untuk train + val vs test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, Y, test_size=0.15, random_state=42, shuffle=True) # 15% untuk test
# Split train_val menjadi train dan val
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42, shuffle=True) # 20% dari sisanya untuk validasi

print(f"Bentuk data pelatihan (X_train, y_train): {X_train.shape}, {y_train.shape}")
print(f"Bentuk data validasi (X_val, y_val): {X_val.shape}, {y_val.shape}")
print(f"Bentuk data uji (X_test, y_test): {X_test.shape}, {y_test.shape}")


Arsitektur Model

In [ ]:
print("\n--- 9. Mendefinisikan Arsitektur Model ---")

# Opsi 1: Model CNN Kustom Anda (dengan penambahan BatchNormalization & Augmentasi)
def build_custom_cnn_model(input_shape, num_classes):
    model = Sequential([
        # Augmentasi Data (On-the-fly) - Ditempatkan di awal model
        # Ini adalah cara yang efisien untuk augmentasi pada GPU
        tf.keras.layers.RandomFlip("horizontal_and_vertical", input_shape=input_shape),
        tf.keras.layers.RandomRotation(0.1), # Rotasi +- 10% dari 360 derajat
        tf.keras.layers.RandomZoom(0.1),    # Zoom in/out +- 10%

        Conv2D(32, (3,3), activation='relu'), # input_shape tidak perlu lagi di sini karena sudah di RandomFlip
        BatchNormalization(),
        MaxPooling2D(2,2),
        
        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        
        Conv2D(128, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        
        Flatten(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='sigmoid') # sigmoid untuk multi-label
    ])
    return model

# Opsi 2: Menggunakan Model Pre-trained (Contoh: MobileNetV2)
def build_transfer_learning_model(input_shape, num_classes, include_augmentation=True):
    # Base model pre-trained
    base_model = MobileNetV2(input_shape=input_shape,
                             include_top=False,
                             weights='imagenet')
    base_model.trainable = False # Bekukan base model awalnya

    inputs = Input(shape=input_shape)
    
    # Tambahkan lapisan augmentasi di sini jika diinginkan
    if include_augmentation:
        x = tf.keras.layers.RandomFlip("horizontal_and_vertical")(inputs)
        x = tf.keras.layers.RandomRotation(0.1)(x)
        x = tf.keras.layers.RandomZoom(0.1)(x)
    else:
        x = inputs

    x = base_model(x, training=False) # Pastikan base model berjalan dalam inference mode saat dibekukan
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    return model

# Pilih model yang akan digunakan
# Jika ingin menggunakan augmentasi pada custom CNN:
# model = build_custom_cnn_model(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), num_classes=len(LABELS_FINAL))

# Jika ingin menggunakan MobileNetV2 dengan augmentasi:
model = build_transfer_learning_model(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), num_classes=len(LABELS_FINAL), include_augmentation=True)

model.summary()